In [ ]:
# --- Step 1: Install dependencies ---
!pip install pandas matplotlib seaborn wordcloud reportlab

# --- Step 2: Imports ---
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from google.colab import files
import datetime

# --- Step 3: Upload ChatGPT Export JSON ---
print("⬆️ Upload your conversations.json file (from ChatGPT export)")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Handle case: file is a list (not wrapped in {"conversations": ...})
if isinstance(data, dict) and "conversations" in data:
    conversations_data = data["conversations"]
elif isinstance(data, list):
    conversations_data = data
else:
    raise ValueError("Unrecognized JSON format")

# --- Step 4: Parse conversations into DataFrame ---
conversations = []
for c in conversations_data:
    conv_id = c.get("id")
    title = c.get("title", "Untitled")
    create_time = None
    if "create_time" in c and c["create_time"]:
        try:
            create_time = datetime.datetime.fromtimestamp(c["create_time"])
        except Exception:
            create_time = None

    messages = c.get("messages", [])
    msg_count = len(messages)
    user_msgs, ai_msgs = 0, 0
    texts = []

    for m in messages:
        role = m.get("author", {}).get("role")
        if role == "user":
            user_msgs += 1
        else:
            ai_msgs += 1

        # Extract message text in all known formats
        content = m.get("content", {})
        if isinstance(content, dict):
            if "parts" in content and isinstance(content["parts"], list):
                texts.extend([str(p) for p in content["parts"] if p])
            elif "text" in content:
                texts.append(str(content["text"]))
        elif isinstance(content, str):
            texts.append(content)

    full_text = " ".join(texts)

    conversations.append({
        "id": conv_id,
        "title": title,
        "date": create_time,
        "messages": msg_count,
        "user_messages": user_msgs,
        "ai_messages": ai_msgs,
        "text": full_text
    })

df = pd.DataFrame(conversations)
df["day"] = pd.to_datetime(df["date"]).dt.date

# --- Step 5: Create Visualizations ---
# Timeline of usage
plt.figure(figsize=(8,4))
df.groupby("day").size().plot(kind="line")
plt.title("Daily ChatGPT Conversations")
plt.xlabel("Date")
plt.ylabel("Conversations")
plt.tight_layout()
plt.savefig("timeline.png")
plt.close()

# Word cloud (safe with fallback)
all_text = " ".join(df["text"].dropna().astype(str))
if not all_text.strip():
    all_text = "ChatGPT Data Empty Placeholder"

wc = WordCloud(width=800, height=400, background_color="white").generate(all_text)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.savefig("wordcloud.png")
plt.close()

# Histogram of message counts
plt.figure(figsize=(6,4))
sns.histplot(df["messages"], bins=20, kde=False)
plt.title("Conversation Lengths (by message count)")
plt.xlabel("Messages per conversation")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("histogram.png")
plt.close()

# --- Step 6: Generate PDF Report ---
styles = getSampleStyleSheet()
doc = SimpleDocTemplate("chatgpt_report.pdf", pagesize=A4)
story = []

story.append(Paragraph("<b>ChatGPT Usage Report</b>", styles["Title"]))
story.append(Spacer(1, 12))
story.append(Paragraph(f"Total Conversations: {len(df)}", styles["Normal"]))
story.append(Paragraph(f"Date Range: {df['day'].min()} → {df['day'].max()}", styles["Normal"]))
story.append(Spacer(1, 12))

story.append(Paragraph("<b>Timeline</b>", styles["Heading2"]))
story.append(Image("timeline.png", width=400, height=200))
story.append(Spacer(1, 12))

story.append(Paragraph("<b>Word Cloud</b>", styles["Heading2"]))
story.append(Image("wordcloud.png", width=400, height=200))
story.append(Spacer(1, 12))

story.append(Paragraph("<b>Conversation Length Distribution</b>", styles["Heading2"]))
story.append(Image("histogram.png", width=400, height=200))
story.append(Spacer(1, 12))

doc.build(story)
print("✅ PDF report created: chatgpt_report.pdf")

# --- Step 7: Download PDF ---
files.download("chatgpt_report.pdf")


⬆️ Upload your conversations.json file (from ChatGPT export)


Saving conversations.json to conversations (5).json
✅ PDF report created: chatgpt_report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 0) Setup
!pip -q install pandas spacy python-dateutil scikit-learn networkx
# Optional: large model (comment if slow)
!python -m spacy download en_core_web_sm

import json, re, math, hashlib, pandas as pd, networkx as nx
from dateutil import parser as dtp
import spacy
nlp = spacy.load("en_core_web_sm")

SRC_JSON = "conversations.json"
NODES_CSV = "nodes.csv"
EDGES_CSV = "edges.csv"

# 1) Load & flatten messages from ChatGPT-style export
def iter_messages(obj):
    # Accepts OpenAI-style export where each conversation has a mapping of messages
    # Adjust paths here if your export schema differs
    if isinstance(obj, dict) and 'mapping' in obj:
        for mid, node in obj['mapping'].items():
            msg = node.get('message')
            if msg and msg.get('content', {}).get('content_type') == 'text':
                text_parts = msg['content'].get('parts') or []
                text = "\n".join([p for p in text_parts if isinstance(p, str)])
                role = msg.get('author', {}).get('role', 'unknown')
                ts = msg.get('create_time') or node.get('create_time') or None
                yield {
                    'message_id': mid,
                    'role': role,
                    'text': text,
                    'timestamp': ts
                }
    # Fallback: try list of conversations
    if isinstance(obj, list):
        for conv in obj:
            yield from iter_messages(conv)

def load_messages(path):
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read()
    try:
        data = json.loads(raw)
    except Exception:
        # Some exports are JSONL; try per-line
        data = [json.loads(line) for line in raw.splitlines() if line.strip()]
    msgs = list(iter_messages(data))
    df = pd.DataFrame(msgs)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
    df['text'] = df['text'].fillna('')
    return df

df = load_messages(SRC_JSON)
print('Loaded messages:', len(df))

# 2) Lightweight NER + keyword tagging
def hash_id(s):
    return hashlib.sha1(s.encode('utf-8')).hexdigest()[:16]

entity_rows = []
edge_rows = []
message_nodes = []

for _, r in df.iterrows():
    mid = r['message_id']
    txt = r['text'][:5000]  # cap for speed
    doc = nlp(txt)
    ents = [(e.text.strip(), e.label_) for e in doc.ents if e.text.strip()]
    # create message node
    message_nodes.append({
        'node_id': f'msg_{mid}',
        'type': 'message',
        'label': (txt[:80] + '...') if len(txt) > 80 else txt,
        'role': r['role'],
        'ts': r['timestamp'].isoformat() if pd.notnull(r['timestamp']) else ''
    })
    # entity nodes + edges
    for etext, etype in ents:
        canon = re.sub(r'\\s+', ' ', etext).strip().lower()
        eid = f'ent_{hash_id(canon + etype)}'
        entity_rows.append({'node_id': eid, 'type': etype, 'label': etext, 'ts': '', 'role': ''})
        edge_rows.append({'src': f'msg_{mid}', 'dst': eid, 'rel': 'mentions', 'weight': 1.0, 'ts': message_nodes[-1]['ts']})

# 3) Build thread/reply edges heuristically by adjacency
for i in range(1, len(df)):
    prev = df.iloc[i-1]['message_id']
    cur = df.iloc[i]['message_id']
    edge_rows.append({'src': f'msg_{prev}', 'dst': f'msg_{cur}', 'rel': 'next_in_thread', 'weight': 0.2, 'ts': ''})

nodes_df = pd.DataFrame(message_nodes + entity_rows).drop_duplicates(subset=['node_id'])
edges_df = pd.DataFrame(edge_rows)

# 4) Compute centrality on message subgraph for sizing
G = nx.DiGraph()
G.add_nodes_from(nodes_df['node_id'])
G.add_edges_from([(e['src'], e['dst']) for _, e in edges_df.iterrows()])
deg = nx.degree_centrality(G)
nodes_df['degree'] = nodes_df['node_id'].map(deg).fillna(0)

nodes_df.to_csv(NODES_CSV, index=False)
edges_df.to_csv(EDGES_CSV, index=False)
print('Wrote', NODES_CSV, EDGES_CSV)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Loaded messages: 6611
Wrote nodes.csv edges.csv
